In [10]:
# Descargado por tiles
import pandas as pd, geopandas as gpd


ethiopia_0 = gpd.read_file("../Data/gjson/adm_0.geojson")
summary_csv = "../Data/csvs/buildings/etiopia_google.csv"
test = pd.read_csv("../Data/csvs/buildings/each_google/161_buildings.csv.gz")
test.head()

,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
0,13.074126,40.061882,40.3476,0.8539,"POLYGON((40.0619234917039 13.0741277003795, 40...",7H5233F6+MQ22
1,12.783786,42.447310,8.6629,0.6743,"POLYGON((42.4473271996833 12.7837770591001, 42...",7H44QCMW+GW7H
2,12.737591,42.193614,60.0961,0.6678,"POLYGON((42.1936645392103 12.7376086239176, 42...",7H44P5QV+2CQP
3,13.255789,40.003832,77.7844,0.8973,"POLYGON((40.0038952882654 13.2557788025164, 40...",7H527243+8G8G
4,13.030569,42.742806,55.5352,0.7151,"POLYGON((42.7428418349994 13.0305361266309, 42...",7H542PJV+64FQ


In [5]:
import os

path = "../Data/csvs/buildings/each_google"
csvs = os.listdir(path)
path_csv = [os.path.join(path, csv) for csv in csvs]
path_csv

['../Data/csvs/buildings/each_google\\161_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\163_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\165_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\167_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\169_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\16f_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\171_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\177_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\179_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\17b_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\17d_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\17f_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\183_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\3d5_buildings.csv.gz',
 '../Data/csvs/buildings/each_google\\3d9_buildings.csv.gz']

In [6]:
gdf_test = gpd.GeoDataFrame(
    test, geometry=gpd.points_from_xy(test["longitude"], test["latitude"])
).set_crs(ethiopia_0.crs)
gdf_test.head()

,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
0,13.074126,40.061882,40.3476,0.8539,POINT (40.06188 13.07413),7H5233F6+MQ22
1,12.783786,42.447310,8.6629,0.6743,POINT (42.44731 12.78379),7H44QCMW+GW7H
2,12.737591,42.193614,60.0961,0.6678,POINT (42.19361 12.73759),7H44P5QV+2CQP
3,13.255789,40.003832,77.7844,0.8973,POINT (40.00383 13.25579),7H527243+8G8G
4,13.030569,42.742806,55.5352,0.7151,POINT (42.74281 13.03057),7H542PJV+64FQ


In [7]:
rows_inside = gpd.sjoin(gdf_test, ethiopia_0, predicate="within")[test.columns]

In [8]:
type(rows_inside)

geopandas.geodataframe.GeoDataFrame

pasos, para un csv

- leer el csv
- convertir el csv en geopandas con el crs del shapefile
- filtrar las observaciones dentro del area de etiopia


In [96]:
def gen_result(
    csv: str = None,
    areas: pd.DataFrame = None,
    target_shp: gpd.GeoDataFrame = ethiopia_0,
    x: str = "longitude",
    y: str = "latitude",
):
    if areas is not None:
        minx, miny, maxx, maxy = target_shp.bounds.values[0]
        areas = areas[(areas[x] >= minx) & (areas[x] <= maxx)]
        areas = areas[(areas[y] >= miny) & (areas[y] <= maxy)]
    else:
        areas = pd.read_csv(csv)

    gdf_areas = gpd.GeoDataFrame(
        areas, geometry=(gpd.points_from_xy(areas[x], areas[y]))

    ).set_crs(target_shp.crs)


    points_in_area = gpd.sjoin(gdf_areas, target_shp, predicate="within")[areas.columns]

    return points_in_area



gen_result(path_csv[0])

,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
0,13.074126,40.061882,40.3476,0.8539,POINT (40.06188 13.07413),7H5233F6+MQ22
2,12.737591,42.193614,60.0961,0.6678,POINT (42.19361 12.73759),7H44P5QV+2CQP
3,13.255789,40.003832,77.7844,0.8973,POINT (40.00383 13.25579),7H527243+8G8G
6,13.964573,40.009031,20.8228,0.7604,POINT (40.00903 13.96457),7H52X275+RJFR
10,12.413394,42.182166,56.0063,0.8201,POINT (42.18217 12.41339),7H44C57J+9V3M
...,...,...,...,...,...,...
61992,13.284292,41.484212,5.4518,0.6593,POINT (41.48421 13.28429),7H537FMM+PM8Q
61994,13.038956,40.074284,15.7407,0.6634,POINT (40.07428 13.03896),7H5223QF+HPM6
61996,13.235213,41.588910,16.5454,0.7377,POINT (41.58891 13.23521),7H536HPQ+3HMC
61997,13.204687,40.861450,68.7752,0.7984,POINT (40.86145 13.20469),7H526V36+VHGF


In [25]:
# %%time
# ethiopia_areas = pd.DataFrame()
# for _csv in path_csv:
#     df_csv = gen_result(_csv)
#     ethiopia_areas = pd.concat((ethiopia_areas, df_csv))
# ethiopia_areas.to_csv(summay_csv, index=False)

CPU times: total: 18min 26s
Wall time: 18min 36s


# Summary


In [117]:
from types_data.utils import cols_dataref

admin3 = gpd.read_file("../Data/gjson/adm_3.geojson")
data_summary = pd.read_csv(summary_csv)

In [118]:
sample = admin3.sample(1)
result = gen_result(areas=data_summary, target_shp=sample)
result

,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
2873064,9.623165,39.105314,66.8042,0.8530,POINT (39.10531 9.62317),6GXXJ4F4+748J
2873310,9.557877,38.866945,15.8644,0.6945,POINT (38.86694 9.55788),6GXWHV58+5Q42
2873781,9.581857,38.715057,55.9154,0.7853,POINT (38.71506 9.58186),6GXWHPJ8+P2V9
2874113,9.609151,38.856190,29.6533,0.8198,POINT (38.85619 9.60915),6GXWJV54+MF82
2874400,9.657223,39.115910,12.5193,0.6818,POINT (39.11591 9.65722),6GXXM448+V9MR
...,...,...,...,...,...,...
19170568,9.571614,38.785592,61.4600,0.8565,POINT (38.78559 9.57161),6GXWHQCP+J6WH
19170762,9.718180,39.032210,8.2626,0.6523,POINT (39.03221 9.71818),6GXXP29J+7VG8
19170785,9.534640,38.751450,39.9459,0.8109,POINT (38.75145 9.53464),6GXWGQM2+VH4M
19171642,9.618899,39.044302,63.6099,0.7590,POINT (39.04430 9.61890),6GXXJ29V+HP3W


In [95]:
minx, miny, maxx, maxy = sample.bounds.values[0]
x, y = 'longitude', 'latitude'
a = data_summary[(data_summary[x] > minx) & (data_summary[x] < maxx)]
# a
# a[(a[y] > miny) & (a[y] < maxy)]

In [119]:
import numpy as np

origen = admin3.sample(1)[cols_dataref]
metrics = [np.mean, np.std, np.sum]
origen

,id,fnid,parent_id,admin_0,admin_1,admin_2,admin_3
336,223132,ET2023A3031120,222718,Ethiopia,Amhara,Central Gondar,Tach Armacho


In [120]:
col_values = "area_in_meters"
min_confidence = 0.65
values = result.query("confidence > @min_confidence")[col_values]
stats = [metric(values) for metric in metrics]
stats

[54.15465132053962, 40.845214642318986, 3420191.1588000003]

In [121]:
new_col_name = "area_google"
(
    origen[f"{new_col_name}_mean"],
    origen[f"{new_col_name}_sd"],
    origen[f"{new_col_name}_sum"],
) = stats
origen

,id,fnid,parent_id,admin_0,admin_1,admin_2,admin_3,area_google_mean,area_google_sd,area_google_sum
336,223132,ET2023A3031120,222718,Ethiopia,Amhara,Central Gondar,Tach Armacho,54.154651,40.845215,3.420191e+06


In [122]:
import warnings
warnings.filterwarnings('ignore')
def get_ind(
    targe_shp3=gpd.GeoDataFrame,
    points_google: pd.DataFrame = data_summary,
    new_col_name=" area_google",
    id_cosl = cols_dataref,
    metrics = [np.mean, np.std, np.sum],
    min_values = 0.65,
    col_values_ref = 'area_in_meters',
    ):
    result_df = gen_result(areas=points_google, target_shp=targe_shp3)
    origen_df = targe_shp3[id_cosl]
    values_result = result_df.query("confidence >= @min_values")[col_values_ref]
    stats_values = [metric(values_result) for metric in metrics]
    (
        origen_df[f"{new_col_name}_mean"],
        origen_df[f"{new_col_name}_sd"],
        origen_df[f"{new_col_name}_sum"],
    ) = stats_values
    return origen_df


result_df_ind = pd.DataFrame()

from tqdm import tqdm
for row in tqdm(range(len(admin3))):
    result = get_ind(admin3.iloc[row:row+1])
    result_df_ind = pd.concat((result_df_ind, result))
    


  0%|          | 0/1141 [00:00<?, ?it/s]

100%|██████████| 1141/1141 [08:30<00:00,  2.24it/s]


In [127]:
result_df_ind.dropna().to_csv("../output/google_areas/google_metrics.csv", index=False)